### News API

In [1]:
nifty_top_tickers = [
    "RELIANCE",
    "HDFCBANK",
    "TCS",
    "BHARTIARTL",
    "ICICIBANK",
    "SBIN",
    "INFY",
    "KOTAKBANK",
    "BAJFINANCE",
    "HINDUNILVR"
]


In [4]:
import s3fs
import pandas as pd
for tick in nifty_top_tickers:
    uri = f"s3://desiquant/data/news/{tick}.parquet.gz"

    s3_params = {
    "endpoint_url": "https://cbabd13f6c54798a9ec05df5b8070a6e.r2.cloudflarestorage.com",
    "key": "5c8ea9c516abfc78987bc98c70d2868a", # FREE credentials for public access!
    "secret": "0cf64f9f0b64f6008cf5efe1529c6772daa7d7d0822f5db42a7c6a1e41b3cadf", # FREE credentials for public access!
    "client_kwargs": {
        "region_name": "auto"
        },
    }
  
    df = pd.read_parquet(uri, storage_options=s3_params)
    df.to_csv(f'Stock News/{tick}_news.csv', index=False)
    print(f"Saved {tick} news data - {len(df)} rows")

Saved RELIANCE news data - 6654 rows
Saved HDFCBANK news data - 8481 rows
Saved TCS news data - 8277 rows
Saved BHARTIARTL news data - 9247 rows
Saved ICICIBANK news data - 6319 rows
Saved SBIN news data - 10000 rows
Saved INFY news data - 8700 rows
Saved KOTAKBANK news data - 3047 rows
Saved BAJFINANCE news data - 2716 rows
Saved HINDUNILVR news data - 3517 rows


### Candles API

In [5]:
import s3fs
import pandas as pd

for tick in nifty_top_tickers:
    uri = f"s3://desiquant/data/candles/{tick}/EQ.parquet.gz"

    s3_params = {
    "endpoint_url": "https://cbabd13f6c54798a9ec05df5b8070a6e.r2.cloudflarestorage.com",
    "key": "5c8ea9c516abfc78987bc98c70d2868a", # FREE credentials for public access!
    "secret": "0cf64f9f0b64f6008cf5efe1529c6772daa7d7d0822f5db42a7c6a1e41b3cadf", # FREE credentials for public access!
    "client_kwargs": {
        "region_name": "auto"
        },
    }

    df = pd.read_parquet(uri, storage_options=s3_params)
    df.to_csv(f'Stock Data/{tick}_candles.csv', index=False)
    print(f"Saved {tick} candles data - {len(df)} rows")

Saved RELIANCE candles data - 604670 rows
Saved HDFCBANK candles data - 604604 rows
Saved TCS candles data - 604675 rows
Saved BHARTIARTL candles data - 604689 rows
Saved ICICIBANK candles data - 604614 rows
Saved SBIN candles data - 604696 rows
Saved INFY candles data - 604622 rows
Saved KOTAKBANK candles data - 604682 rows
Saved BAJFINANCE candles data - 604687 rows
Saved HINDUNILVR candles data - 604603 rows
